# Question-3: PACC

## Title
Enhancing Pattern Accuracy Metric in Candlestick Pattern Mining for Stock Price Forecasting

## Abstract

Predicting future stock prices is notoriously challenging due to the influence of numerous variables. However, recent advances in pattern mining and machine learning have yielded promising results in forecasting stock market behavior. Candlestick patterns which are visual representations of stock price movements can be used to identify patterns that may indicate future trends.

Current prediction methods rely on matching sequences of candlesticks to patterns with the highest "pattern accuracy" scores. However, this metric has limitations, as it can yield high scores even when patterns appear only once in historical data, reducing reliability. This project aims to improve predictive accuracy by developing a weighted pattern accuracy metric that considers the frequency and significance of patterns within historical data, ultimately enhancing the robustness of stock price forecasts.




